In [1]:
import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片
import numpy as np
import time

from PIL import Image
from PIL import ImageDraw

item = 0
def next_batch(x,y,batch_size):
    global item
    while True:
        item += batch_size
        if item+2*batch_size > x.shape[0]:   item = 0
        return x[item:item+batch_size,:],y[item:item+batch_size,:]


def load_img(path= "D:\\Unistar\\TV Logo Recognition\\Logo Data\\cctv5+",num = 10,label = 0,total_label = 3):
    a_s = []
    for i in range(num):
        file = path+ "/video_out_sd_{}.yuv".format(i)

        f = open(file,'rb')
        buf = f.read()
        buf = np.frombuffer(buf, dtype=np.uint8)
        buf = buf.reshape([1,96*35])
        if i == 0:    matrix = buf
        else:         matrix = np.concatenate([matrix,buf],axis = 0)
        a_s.append(buf)
        #lena =  Image.frombuffer('L',(96,35),buf)
        #lena.show()
        #time.sleep(1)
        f.close()

    #matrix = np.array(a_s)
    
    #print(matrix.shape)
    label_s = np.zeros( (matrix.shape[0],total_label),dtype = np.float32 )
    label_s[:,label] = 1
    return matrix,label_s

def showImgfrombytes(bytestream,shape = (96,35)):
    lena =  Image.frombuffer('L',shape,bytestream)
    lena = lena.transpose(Image.FLIP_TOP_BOTTOM) #上下互换
    lena.show()
    #draw = ImageDraw.Draw(img)
    

def listmapping(channel_list,LOGO_s):
    out_s = []
    for i in channel_list:
        out_s.append(LOGO_s[int(i)][0])
    return out_s
    
    
TV_LOGO_s = [('cctv5+',1022),('cctv1hd',1044),('jswshd',1021),('dfwshd',1001),('cctv5',1001),
             ('zjwshd',1001),('cctv1',1001),('bjwshd',1001),('hnwshd',1001),('cctv5hd',1001)]
CLASSES = len(TV_LOGO_s)
LOGOPATH = "D:\\Unistar\\TV Logo Recognition\\Logo Data 96x35\\"
img_s   = [0]*CLASSES
label_s = [0]*CLASSES
for i in range(CLASSES):
    img_s[i],label_s[i] = load_img(LOGOPATH+TV_LOGO_s[i][0],int(TV_LOGO_s[i][1]),i,CLASSES)


img   = img_s[0]
label = label_s[0]
for i in range(1,len(img_s)):
    img   = np.concatenate([img,img_s[i]],axis = 0)
    label = np.concatenate([label,label_s[i]],axis = 0)


In [2]:
permutation = np.random.permutation(img.shape[0])
shuffled_img = img[permutation, :]
shuffled_labels = label[permutation, :]

total_num = img.shape[0]
train_num = int(0.8*total_num)

img_train   = shuffled_img[0:train_num,:]
label_train = shuffled_labels[0:train_num,:]
img_test    = shuffled_img[train_num:,:]
label_test  = shuffled_labels[train_num:,:]

print(img_train.shape, label_train.shape)
print(img_test.shape, label_test.shape)



(8075, 3360) (8075, 10)
(2019, 3360) (2019, 10)


In [ ]:
print(img_train.dtype,label_train.dtype)

In [ ]:
showImgfrombytes(img_test[1550])
print(label_test[1550])

In [3]:
import numpy as np
import tensorflow as tf
#import tensorflow.examples.tutorials.mnist.input_data as input_data
#mnist = input_data.read_data_sets(r"D:\MachineLearning\minst", one_hot=True)

def batch_normal(Y,offset_Beta,scale_J,iteration):
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.998, iteration) # adding the iteration prevents from averaging across non-existing iterations
    mean, variance = tf.nn.moments(Y, [0])
    update_moving_averages = exp_moving_avg.apply([mean, variance])
    m = exp_moving_avg.average(mean)
    v = exp_moving_avg.average(variance)
   # Ybn = tf.nn.batch_normalization(Y, m, v, offset_Beta, scale_J, 1e-5)
    Ybn = tf.nn.batch_normalization(Y, mean, variance, offset_Beta, scale_J, 1e-5)
    return Ybn,update_moving_averages


def hidden_layer(X, sizeOutput, iteration = 10, non_linear_name = '',enable_bn=False):
    sizeInput = X.shape[1]
    W = tf.Variable( tf.truncated_normal([int(sizeInput),int(sizeOutput)],stddev=0.001) )
    
    if enable_bn:   
        Z = tf.matmul(X,W)         #In bach norm, the bias B can be ommited due to the offset
        scale_J = tf.Variable(tf.ones([int(sizeOutput)]))
        offset_Beta = tf.Variable(tf.zeros([int(sizeOutput)]))
        Y, update_moving_averages = batch_normal(Z,offset_Beta,scale_J, iteration)
    else:           
        B = tf.Variable(tf.zeros([1,sizeOutput]))
        Y = tf.matmul(X,W) + B
        
        
    if non_linear_name == '':             return   Y,0
    elif non_linear_name == 'softmax':    A = tf.nn.softmax(Y)
    elif non_linear_name == 'relu':       A = tf.nn.relu(Y)
    elif non_linear_name == 'sigmoid':    A = tf.nn.sigmoid(Y)
    return A, update_moving_averages


X       = tf.placeholder(tf.float32, [None, 96*35],name = 'Input')
Y_LABEL = tf.placeholder(tf.float32, [None, CLASSES], name = 'Label')
lr      = tf.placeholder(tf.float32, name = 'LearningRate')
iters   = tf.placeholder(tf.int32, name = 'Iterations')

A0, update_ema0 = hidden_layer(X,512, iters, 'relu',True)
A1, update_ema1 = hidden_layer(A0,256, iters, 'relu',True)
A2, update_ema2 = hidden_layer(A1,128,iters, 'relu',True)
A3, update_ema3 = hidden_layer(A2,64, iters, 'relu',True)
A4, update_ema4 = hidden_layer(A3,32, iters, 'relu',True)
Y_linear,nocare = hidden_layer(A4, Y_LABEL.shape[1]) 
Y_predict       = tf.nn.softmax(Y_linear,name = 'Y_predict')
update_ema = tf.group(update_ema0, update_ema1, update_ema2, update_ema3, update_ema4)
#update_ema = tf.group(update_ema0, update_ema2, update_ema3)

#cross_entropy = tf.reduce_mean(
#    tf.nn.softmax_cross_entropy_with_logits(labels=Y_LABEL, logits=Y_linear))
cross_entropy = tf.reduce_mean( -tf.reduce_sum(Y_LABEL * tf.log(Y_predict), reduction_indices=[1]) )
#train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()


max_learning_rate = 0.1
min_learning_rate = 0.001
decay_speed = 2000.0


saver = tf.train.Saver()
img_train_float = np.float32(img_train/255)
img_test_float = np.float32(img_test/255)
print(img_train_float.shape)
for _ in range(2000):
    batch_xs, batch_ys= next_batch(img_train_float,label_train,500)
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * np.exp(-_/decay_speed)
    sess.run(train_step, feed_dict={X: batch_xs, Y_LABEL: batch_ys, lr:learning_rate})
    if _%100 == 0:
        correct_prediction = tf.equal(tf.argmax(Y_linear, 1), tf.argmax(Y_LABEL, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        #print('This is round {} ...'.format(_))
        #print(batch_ys[0])
        #showImgfrombytes(batch_xs[0])
        saver.save(sess, "./Model/TVLogo_5_relu_softmax.ckpt",global_step= _ )
        print(sess.run(accuracy, feed_dict={X:img_test_float, Y_LABEL: label_test}))

(8075, 3360)
0.130758
0.995047
0.998019
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009
0.999009


In [ ]:
saver = tf.train.Saver() 
saver.save(sess, "./Model/TVLogo_5_relu_softmax.ckpt")  

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, "./Model/TVLogo_5_relu_softmax.ckpt")
print(sess.run(tf.get_default_graph().get_tensor_by_name("train_step")))

In [4]:
import tensorflow as tf
with tf.Session() as sess:

    #初始化变量
    sess.run(tf.global_variables_initializer())

    #获取最新的checkpoint，其实就是解析了checkpoint文件
    latest_ckpt = tf.train.latest_checkpoint("./Model")

    #加载图
    restore_saver = tf.train.import_meta_graph('./Model/TVLogo_5_relu_softmax.ckpt-1900.meta')

    #恢复图，即将weights等参数加入图对应位置中
    restore_saver.restore(sess, latest_ckpt)

    #将图中的变量转为常量
    output_graph_def = tf.graph_util.convert_variables_to_constants(
        sess, sess.graph_def , ["Y_predict"])
    #将新的图保存到"/pretrained/graph.pb"文件中
    tf.train.write_graph(output_graph_def, 'pretrained', "graph.pb", as_text=False)

INFO:tensorflow:Restoring parameters from ./Model\TVLogo_5_relu_softmax.ckpt-1900
INFO:tensorflow:Froze 17 variables.
Converted 17 variables to const ops.


In [5]:
constant_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def,['Y_predict'])
with tf.gfile.FastGFile('./pretrained/graph.pb', mode='wb') as f:
    f.write(constant_graph.SerializeToString())

RuntimeError: Attempted to use a closed Session.

In [ ]:
import tensorflow as tf
import numpy as np
with tf.Session() as sess:
    with open('./pretrained/graph.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
        img_tmp = img_test[0:25]
        test_input = np.float32(img_tmp)
        #test_input = np.reshape(test_input, [1,96*35])
        #print(test_input)
        #test_input = np.zeros([1,96*35],dtype = np.float32)
        #for i in range(5):    showImgfrombytes(img_tmp[i])
        print(test_input.shape)
        output = tf.import_graph_def(graph_def, input_map={'Input:0':test_input}, return_elements=['Y_predict:0'])
        prediction = sess.run(output)
        #print(prediction)
        print(np.argmax(prediction[0],1))
        

In [6]:
def recognize(img_input, pb_file_path):
    with tf.Graph().as_default():
        output_graph_def = tf.GraphDef()

        with open(pb_file_path, "rb") as f:
            output_graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(output_graph_def, name="")

        with tf.Session() as sess:
            init = tf.global_variables_initializer()
            sess.run(init)

            input_x = sess.graph.get_tensor_by_name("Input:0")
            # print input_x
            out_softmax = sess.graph.get_tensor_by_name("Y_predict:0")
            # print out_softmax
            # out_label = sess.graph.get_tensor_by_name("output:0")
            # print out_label

            # img = io.imread(jpg_path)
            # img = transform.resize(img, (224, 224, 3))
            
            img_try = img_input
            #showImgfrombytes(img[7])
            #showImgfrombytes(img_try[1])
            #showImgfrombytes(img[2])
            test_input = np.float32(img_try/255)
            #test_input = np.reshape(test_input/255, [1,96*35])
            #print(test_input)
            img_out_softmax = sess.run(out_softmax, feed_dict={input_x:test_input})

            #print ("img_out_softmax:",img_out_softmax)
            prediction_labels = np.argmax(img_out_softmax, axis=1)
            #print ("label:",prediction_labels)
            return prediction_labels
            #print('true label:',mnist.test.labels[0])

#r = recognize(img_test, "./ResNet_L152_retrain/pb_model/frozen_model_ResNet_L152.pb")
#print(listmapping(r,TV_LOGO_s))
start = 10
for i in range(1,10):
    r = recognize(img_test[start:start+i], "./pretrained/graph.pb")
    print(listmapping(r,TV_LOGO_s))
showImgfrombytes(img_test[start])
showImgfrombytes(img_test[start+1])



['cctv5+']
['cctv5hd', 'hnwshd']
['cctv1', 'jswshd', 'cctv1hd']
['cctv1', 'jswshd', 'cctv1hd', 'cctv5']
['cctv1', 'jswshd', 'cctv1hd', 'cctv5', 'zjwshd']
['cctv1', 'jswshd', 'cctv1hd', 'cctv5', 'zjwshd', 'dfwshd']
['cctv1', 'cctv5+', 'cctv5hd', 'cctv5', 'zjwshd', 'zjwshd', 'jswshd']
['cctv1', 'jswshd', 'cctv5hd', 'cctv5', 'zjwshd', 'zjwshd', 'jswshd', 'cctv5+']
['cctv1', 'jswshd', 'cctv5hd', 'cctv5', 'zjwshd', 'zjwshd', 'jswshd', 'cctv5+', 'hnwshd']


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: the frombuffer defaults may change in a future release; for portability, change the call to read:
  frombuffer(mode, size, data, 'raw', mode, 0, 1)


In [ ]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
 
#os.environ['CUDA_VISIBLE_DEVICES']='2'  #设置GPU
model_path  = "./Model/TVLogo_5_relu_softmax.ckpt-1900" #设置model的路径
 
def main():
    tf.reset_default_graph()
    saver = tf.train.import_meta_graph("./Model/TVLogo_5_relu_softmax.ckpt-1900.meta")
    #flow = tf.cast(flow, tf.uint8, 'out') #设置输出类型以及输出的接口名字，为了之后的调用pb的时候使用
    with tf.Session() as sess:
        saver.restore(sess, model_path)
        #保存图
        tf.train.write_graph(sess.graph_def, './ResNet_L152_retrain/pb_model', 'model_ResNet_L153.pb')
        #把图和参数结构一起
        freeze_graph.freeze_graph('ResNet_L152_retrain/pb_model/model_ResNet_L153.pb',
                                  '',
                                  False,
                                  model_path,
                                  'Y_predict',
                                  'save/restore_all',
                                  'save/Const:0',
                                  'ResNet_L152_retrain/pb_model/frozen_model_ResNet_L153.pb',
                                  False,
                                  "")
    print("done")
     
if __name__ == '__main__':
    main()

In [7]:
import tensorflow as tf
import numpy as np
with tf.Session() as sess:
    with open('./pretrained/graph.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
        for i in range(10):
            test_input = np.float32(img_train[i]/255)
            test_input = np.reshape(test_input,[1,96*35])
        #test_input = np.float32(img/255)
        #test_input = np.reshape(test_input, [1,96*35])
            print(test_input)
        #test_input = np.zeros([1,96*35],dtype = np.float32)
        #showImgfrombytes(img)
            #print(test_input.shape)
            output = tf.import_graph_def(graph_def, input_map={'Input:0':test_input}, return_elements=['Y_predict:0'])
            prediction = sess.run(output)
            #print(prediction)
            r = np.argmax(prediction[0],1)
            print(listmapping(r,TV_LOGO_s))

[[ 0.02352941  0.02352941  0.02352941 ...,  0.02352941  0.01960784
   0.01960784]]
['cctv5+']
[[ 0.07450981  0.07450981  0.07450981 ...,  0.17647059  0.18039216
   0.18039216]]
['cctv5+']
[[ 0.25882354  0.28235295  0.29803923 ...,  0.37254903  0.34117648
   0.27843139]]
['cctv5+']
[[ 0.13725491  0.25098041  0.52549022 ...,  0.10588235  0.10980392
   0.10980392]]
['cctv5+']
[[ 0.8392157   0.84313726  0.84313726 ...,  0.78039217  0.78039217
   0.78039217]]
['cctv5+']
[[ 0.28627452  0.28627452  0.28627452 ...,  0.39607844  0.38431373
   0.38039216]]
['cctv5+']
[[ 0.36470589  0.36470589  0.36470589 ...,  0.36470589  0.36078432
   0.36078432]]
['cctv5+']
[[ 0.50196081  0.49803922  0.50588238 ...,  0.76078433  0.7647059
   0.7647059 ]]
['cctv5+']
[[ 0.36078432  0.36470589  0.36862746 ...,  0.25490198  0.25490198
   0.25490198]]
['cctv5+']
[[ 0.50196081  0.50196081  0.50980395 ...,  0.75294119  0.75686276
   0.7647059 ]]
['cctv5+']
